In [96]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import json
import urllib
import sys
import time
import io
from PIL import Image
import hashlib

In [97]:
# adding path to geckodriver to the OS environment variable
os.environ["PATH"] += os.pathsep + os.getcwd()

## Configuration

In [12]:
# configuration
download_path = "dataset/"
# Imges
words_to_search = ['seafood', 'clams']  # Mots à chercher
nb_to_download = [10, 5]                # Nb image par catégorie
first_image_position = [25, 0]          # A partir de la combien-t-ième image on télécharge

In [13]:
def main():
    if len(words_to_search) != len(nb_to_download) or len(nb_to_download) != len(first_image_position) :
        raise ValueError('You may have forgotten to configure one of the lists (length is different)')
    i= 0
    # For each word in the list, we download the number of images requested
    while i<len(words_to_search):
        print("Words "+str(i)+" : "+str(nb_to_download[i])+"x\""+words_to_search[i]+"\"")
        if nb_to_download[i] > 0:
            search_and_save(words_to_search[i],nb_to_download[i], first_image_position[i])
        i+=1

## Connexion Google Image

In [14]:
def search_and_save(text, number, first_position):
    # Number_of_scrolls * 400 images will be opened in the browser
    number_of_scrolls = int((number + first_position)/ 400 + 1) 
    print("Search : "+text+" ; number : "+str(number)+"; first_position : "+str(first_position)+" ; scrolls : "+str(number_of_scrolls))

    # Create directories to save images
    if not os.path.exists(download_path + text.replace(" ", "_")):
        os.makedirs(download_path + text.replace(" ", "_"))

    # Connect to Google Image
    url = "https://www.google.co.in/search?q="+text+"&source=lnms&tbm=isch"
    driver = webdriver.Firefox()
    driver.get(url)
    headers = {}
    headers['User-Agent'] = "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"
    extensions = {"jpg", "jpeg", "png", "gif"}
    
    img_count = 0
    downloaded_img_count = 0
    img_skip = 0
    
    # Prepare Google Page
    for _ in range(number_of_scrolls):
        for __ in range(10):
            # Multiple scrolls needed to show all 400 images
            driver.execute_script("window.scrollBy(0, 1000000)")
            time.sleep(0.2)
        # to load next 400 images
        time.sleep(2.5)
        try:
            driver.find_element_by_xpath("//input[@value='Show more results']").click()
            time.sleep(2.5)
        except Exception as e:
            print("Less images found:"+ str(e))
            break

    # Process (download) images
#     imges = driver.find_elements_by_xpath('//div[contains(@class,"rg_meta")]')
    imges = driver.find_elements_by_class_name('rg_i')
    print("Total images:"+ str(len(imges)) + "\n")
    for img in imges:
        if img_skip < first_position:
            # Skip first images if asked to
            img_skip += 1
        else :
            # Get image
            img_count += 1
            print(img.get_attribute('innerHTML'))
#             json.loads(img.get_attribute('innerHTML'))
#             img_url = json.loads(img.get_attribute('innerHTML'))["ou"]
#             img_type = json.loads(img.get_attribute('innerHTML'))["ity"]
#             print("Downloading image "+ str(img_count) + ": "+ img_url)
#             try:
#                 if img_type not in extensions:
#                     img_type = "jpg"
#                 # Download image and save it
#                 req = urllib.request.Request(img_url, headers=headers)
#                 raw_img = urllib.request.urlopen(req).read()
#                 f = open(download_path+text.replace(" ", "_")+"/"+str(img_skip+downloaded_img_count)+"."+img_type, "wb")
#                 f.write(raw_img)
#                 f.close
#                 downloaded_img_count += 1
#             except Exception as e:
#                 print("Download failed:"+ str(e))
#             finally:
#                 print("")
#             if downloaded_img_count >= number:
#                 break

    print("Total skipped : "+str(img_skip)+"; Total downloaded : "+ str(downloaded_img_count)+ "/"+ str(img_count))
    driver.quit()

In [15]:
if __name__ == "__main__":
    main()

Words 0 : 10x"seafood"
Search : seafood ; number : 10; first_position : 25 ; scrolls : 1
Less images found:Message: Element <input class="mye4qd" type="button"> could not be scrolled into view

Total images:400
























































































































































































































































































































































































Total skipped : 25; Total downloaded : 0/375
Words 1 : 5x"clams"
Search : clams ; number : 5; first_position : 0 ; scrolls : 1
Less images found:Message: Element <input class="mye4qd" type="button"> could not be scrolled into view

Total images:400






































































































































































## ==================

In [3]:
# Source : https://dev.to/petercour/image-scraping-with-python-330a
# --------------------------------------------------------------------
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import os

download_path = "dataset/"

def download_baidu_2(word, number_img_to_download): 
    
    url = 'https://image.baidu.com/search/flip?tn=baiduimage&ie=utf-8&word=' + word + '&ct=201326592&v=flip'
    result = requests.get(url)
    html = result.text
    pic_url = re.findall('"objURL":"(.*?)",',html,re.S)

    downloaded_img_count = 0

    for each in pic_url:
        print(pic_url)

        try:
            pic= requests.get(each, timeout=10)
        except requests.exceptions.ConnectionError:
            print ('exception')
            continue

        # Create directories to save images
        if not os.path.exists(download_path + word.replace(" ", "_")):
            os.makedirs(download_path + word.replace(" ", "_"))

        image_name = 'pictures_' + word + '_' + str(downloaded_img_count) + '.jpg'
        file_to_download = download_path + word + "/" + image_name

        fp = open(file_to_download, 'wb')
        fp.write(pic.content)
        fp.close()
        downloaded_img_count += 1

        if downloaded_img_count >= number_img_to_download:
            break
    
def download_baidu(keyword): 
    url = 'https://image.baidu.com/search/flip?tn=baiduimage&ie=utf-8&word='+word+'&ct=201326592&v=flip'
    result = requests.get(url)
    html = result.text
    pic_url = re.findall('"objURL":"(.*?)",',html,re.S)
    i = 0

    for each in pic_url:
        print(pic_url)
        try:
            pic= requests.get(each, timeout=10)
        except requests.exceptions.ConnectionError:
            print ('exception')
            continue

        string = 'pictures_' + keyword + '_'+str(i) + '.jpg'
        
        fp = open(string,'wb')
        fp.write(pic.content)
        fp.close()
        i += 1

def download_google(word):
    url = 'https://www.google.com/search?q=' + word + '&client=opera&hs=cTQ&source=lnms&tbm=isch&sa=X&ved=0ahUKEwig3LOx4PzKAhWGFywKHZyZAAgQ_AUIBygB&biw=1920&bih=982'
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    
    for raw_img in soup.find_all('img'):
        link = raw_img.get('src')
        os.system("wget " + link)

In [6]:
if __name__ == '__main__':
#     word = input("Input key word: ")
    queries = ["clams", "seafood"]
    nb_images_per_keyword = [3, 1000]
    
    for idx, query in enumerate(queries):
        download_baidu_2(query, nb_images_per_keyword[idx])
    
    
#     download_google("clams")

['http://fsvr.dealmoon.com/dealmoon/3e9/ca1/bc4/8c4/d28/119/09e/97f/dd7/d2e/04.jpg', 'http://assets.simplyrecipes.com/wp-content/uploads/2008/07/baked-stuffed-clams-horiz-b-1200.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/02/5b/a6/6c/steamed-clams.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/04/43/ec/7d/spaghetti-clams.jpg', 'http://hbimg.b0.upaiyun.com/01b0e4c51fd801d787a586b3b383a6baf17899bc19890-dcBCeR_fw658', 'http://photocdn.sohu.com/20160119/mp55188076_1453141897671_6.jpeg', 'http://www.the-pasta-project.com/wp-content/uploads/2017/01/36608895_m.jpg', 'http://5b0988e595225.cdn.sohucs.com/q_70,c_zoom,w_640/images/20190107/a185637fc7b948508c3c92588f11db41.jpeg', 'http://5b0988e595225.cdn.sohucs.com/images/20170902/1f73200f6aa3413aaa33df69a5f665dc.jpeg', 'http://www.goldenbaytrading.com/Upload/PicFiles/2015.7.30_11.28.24_6194.jpg', 'http://b-ssl.duitang.com/uploads/item/201203/20/20120320131941_wEn5L.jpeg', 'http://www.gannett-cdn.com/presto/2018/12/13/PCNJ/6fa489

['http://fsvr.dealmoon.com/dealmoon/3e9/ca1/bc4/8c4/d28/119/09e/97f/dd7/d2e/04.jpg', 'http://assets.simplyrecipes.com/wp-content/uploads/2008/07/baked-stuffed-clams-horiz-b-1200.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/02/5b/a6/6c/steamed-clams.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/04/43/ec/7d/spaghetti-clams.jpg', 'http://hbimg.b0.upaiyun.com/01b0e4c51fd801d787a586b3b383a6baf17899bc19890-dcBCeR_fw658', 'http://photocdn.sohu.com/20160119/mp55188076_1453141897671_6.jpeg', 'http://www.the-pasta-project.com/wp-content/uploads/2017/01/36608895_m.jpg', 'http://5b0988e595225.cdn.sohucs.com/q_70,c_zoom,w_640/images/20190107/a185637fc7b948508c3c92588f11db41.jpeg', 'http://5b0988e595225.cdn.sohucs.com/images/20170902/1f73200f6aa3413aaa33df69a5f665dc.jpeg', 'http://www.goldenbaytrading.com/Upload/PicFiles/2015.7.30_11.28.24_6194.jpg', 'http://b-ssl.duitang.com/uploads/item/201203/20/20120320131941_wEn5L.jpeg', 'http://www.gannett-cdn.com/presto/2018/12/13/PCNJ/6fa489

['http://de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/responsive/16:9/de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/restaurants-irs/235318/feast', 'http://res.klook.com/images/fl_lossy.progressive,q_65/c_fill,w_1295,h_720,f_auto/w_80,x_15,y_15,g_south_west,l_klook_water/activities/mawfiuvzvdaxk7lwhsft/RamayanaBuffetatKingPowerDutyFree.jpg', 'http://b3-q.mafengwo.net/s10/M00/66/CF/wKgBZ1kcpt2AO3iRABDLruozamU07.jpeg?imageView2/2/w/680/q/90', 'http://www.szdaily.com/images/attachement/jpg/site542/20140905/d4bed994063f1573650f0b.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/05/49/17/ab/cafe-kitchen-grand-hyatt.jpg', 'http://www.harbour-plaza.com/8degrees/SpecialOfferImages/Sustainable%20Seafood%20Dinner%20Buffet%20%28Jan-Mar%202014%29.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/06/20/4f/b4/sashimi.jpg', 'http://i1.chuimg.com/770a0dc79d0611e5b3dcc5b825b69b9a.jpg@2o_50sh_1pr_1l_600w_90q_1wh', 'http://img1.doubanio.com

['http://de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/responsive/16:9/de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/restaurants-irs/235318/feast', 'http://res.klook.com/images/fl_lossy.progressive,q_65/c_fill,w_1295,h_720,f_auto/w_80,x_15,y_15,g_south_west,l_klook_water/activities/mawfiuvzvdaxk7lwhsft/RamayanaBuffetatKingPowerDutyFree.jpg', 'http://b3-q.mafengwo.net/s10/M00/66/CF/wKgBZ1kcpt2AO3iRABDLruozamU07.jpeg?imageView2/2/w/680/q/90', 'http://www.szdaily.com/images/attachement/jpg/site542/20140905/d4bed994063f1573650f0b.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/05/49/17/ab/cafe-kitchen-grand-hyatt.jpg', 'http://www.harbour-plaza.com/8degrees/SpecialOfferImages/Sustainable%20Seafood%20Dinner%20Buffet%20%28Jan-Mar%202014%29.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/06/20/4f/b4/sashimi.jpg', 'http://i1.chuimg.com/770a0dc79d0611e5b3dcc5b825b69b9a.jpg@2o_50sh_1pr_1l_600w_90q_1wh', 'http://img1.doubanio.com

['http://de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/responsive/16:9/de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/restaurants-irs/235318/feast', 'http://res.klook.com/images/fl_lossy.progressive,q_65/c_fill,w_1295,h_720,f_auto/w_80,x_15,y_15,g_south_west,l_klook_water/activities/mawfiuvzvdaxk7lwhsft/RamayanaBuffetatKingPowerDutyFree.jpg', 'http://b3-q.mafengwo.net/s10/M00/66/CF/wKgBZ1kcpt2AO3iRABDLruozamU07.jpeg?imageView2/2/w/680/q/90', 'http://www.szdaily.com/images/attachement/jpg/site542/20140905/d4bed994063f1573650f0b.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/05/49/17/ab/cafe-kitchen-grand-hyatt.jpg', 'http://www.harbour-plaza.com/8degrees/SpecialOfferImages/Sustainable%20Seafood%20Dinner%20Buffet%20%28Jan-Mar%202014%29.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/06/20/4f/b4/sashimi.jpg', 'http://i1.chuimg.com/770a0dc79d0611e5b3dcc5b825b69b9a.jpg@2o_50sh_1pr_1l_600w_90q_1wh', 'http://img1.doubanio.com

...

['http://de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/responsive/16:9/de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/restaurants-irs/235318/feast', 'http://res.klook.com/images/fl_lossy.progressive,q_65/c_fill,w_1295,h_720,f_auto/w_80,x_15,y_15,g_south_west,l_klook_water/activities/mawfiuvzvdaxk7lwhsft/RamayanaBuffetatKingPowerDutyFree.jpg', 'http://b3-q.mafengwo.net/s10/M00/66/CF/wKgBZ1kcpt2AO3iRABDLruozamU07.jpeg?imageView2/2/w/680/q/90', 'http://www.szdaily.com/images/attachement/jpg/site542/20140905/d4bed994063f1573650f0b.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/05/49/17/ab/cafe-kitchen-grand-hyatt.jpg', 'http://www.harbour-plaza.com/8degrees/SpecialOfferImages/Sustainable%20Seafood%20Dinner%20Buffet%20%28Jan-Mar%202014%29.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/06/20/4f/b4/sashimi.jpg', 'http://i1.chuimg.com/770a0dc79d0611e5b3dcc5b825b69b9a.jpg@2o_50sh_1pr_1l_600w_90q_1wh', 'http://img1.doubanio.com

['http://de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/responsive/16:9/de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/restaurants-irs/235318/feast', 'http://res.klook.com/images/fl_lossy.progressive,q_65/c_fill,w_1295,h_720,f_auto/w_80,x_15,y_15,g_south_west,l_klook_water/activities/mawfiuvzvdaxk7lwhsft/RamayanaBuffetatKingPowerDutyFree.jpg', 'http://b3-q.mafengwo.net/s10/M00/66/CF/wKgBZ1kcpt2AO3iRABDLruozamU07.jpeg?imageView2/2/w/680/q/90', 'http://www.szdaily.com/images/attachement/jpg/site542/20140905/d4bed994063f1573650f0b.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/05/49/17/ab/cafe-kitchen-grand-hyatt.jpg', 'http://www.harbour-plaza.com/8degrees/SpecialOfferImages/Sustainable%20Seafood%20Dinner%20Buffet%20%28Jan-Mar%202014%29.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/06/20/4f/b4/sashimi.jpg', 'http://i1.chuimg.com/770a0dc79d0611e5b3dcc5b825b69b9a.jpg@2o_50sh_1pr_1l_600w_90q_1wh', 'http://img1.doubanio.com

['http://de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/responsive/16:9/de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/restaurants-irs/235318/feast', 'http://res.klook.com/images/fl_lossy.progressive,q_65/c_fill,w_1295,h_720,f_auto/w_80,x_15,y_15,g_south_west,l_klook_water/activities/mawfiuvzvdaxk7lwhsft/RamayanaBuffetatKingPowerDutyFree.jpg', 'http://b3-q.mafengwo.net/s10/M00/66/CF/wKgBZ1kcpt2AO3iRABDLruozamU07.jpeg?imageView2/2/w/680/q/90', 'http://www.szdaily.com/images/attachement/jpg/site542/20140905/d4bed994063f1573650f0b.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/05/49/17/ab/cafe-kitchen-grand-hyatt.jpg', 'http://www.harbour-plaza.com/8degrees/SpecialOfferImages/Sustainable%20Seafood%20Dinner%20Buffet%20%28Jan-Mar%202014%29.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/06/20/4f/b4/sashimi.jpg', 'http://i1.chuimg.com/770a0dc79d0611e5b3dcc5b825b69b9a.jpg@2o_50sh_1pr_1l_600w_90q_1wh', 'http://img1.doubanio.com

exception
['http://de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/responsive/16:9/de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/restaurants-irs/235318/feast', 'http://res.klook.com/images/fl_lossy.progressive,q_65/c_fill,w_1295,h_720,f_auto/w_80,x_15,y_15,g_south_west,l_klook_water/activities/mawfiuvzvdaxk7lwhsft/RamayanaBuffetatKingPowerDutyFree.jpg', 'http://b3-q.mafengwo.net/s10/M00/66/CF/wKgBZ1kcpt2AO3iRABDLruozamU07.jpeg?imageView2/2/w/680/q/90', 'http://www.szdaily.com/images/attachement/jpg/site542/20140905/d4bed994063f1573650f0b.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/05/49/17/ab/cafe-kitchen-grand-hyatt.jpg', 'http://www.harbour-plaza.com/8degrees/SpecialOfferImages/Sustainable%20Seafood%20Dinner%20Buffet%20%28Jan-Mar%202014%29.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/06/20/4f/b4/sashimi.jpg', 'http://i1.chuimg.com/770a0dc79d0611e5b3dcc5b825b69b9a.jpg@2o_50sh_1pr_1l_600w_90q_1wh', 'http://img1.do

['http://de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/responsive/16:9/de8c9603ecbba0bd9348-dc0eab60e8c54b9a69ccd4523eb3f0ba.r34.cf1.rackcdn.com/restaurants-irs/235318/feast', 'http://res.klook.com/images/fl_lossy.progressive,q_65/c_fill,w_1295,h_720,f_auto/w_80,x_15,y_15,g_south_west,l_klook_water/activities/mawfiuvzvdaxk7lwhsft/RamayanaBuffetatKingPowerDutyFree.jpg', 'http://b3-q.mafengwo.net/s10/M00/66/CF/wKgBZ1kcpt2AO3iRABDLruozamU07.jpeg?imageView2/2/w/680/q/90', 'http://www.szdaily.com/images/attachement/jpg/site542/20140905/d4bed994063f1573650f0b.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/05/49/17/ab/cafe-kitchen-grand-hyatt.jpg', 'http://www.harbour-plaza.com/8degrees/SpecialOfferImages/Sustainable%20Seafood%20Dinner%20Buffet%20%28Jan-Mar%202014%29.jpg', 'http://media-cdn.tripadvisor.com/media/photo-s/06/20/4f/b4/sashimi.jpg', 'http://i1.chuimg.com/770a0dc79d0611e5b3dcc5b825b69b9a.jpg@2o_50sh_1pr_1l_600w_90q_1wh', 'http://img1.doubanio.com

In [9]:
# Source : https://gitpress.io/c/1452/selenium-baidu_image_spider
# --------------------------------------------------------------------
import time
import datetime
from selenium import webdriver
from selenium.common.exceptions import *

# adding path to geckodriver to the OS environment variable
os.environ["PATH"] += os.pathsep + os.getcwd()

# Firefox
driver = webdriver.Firefox()
print('Le navigateur est lancé')


# Set timeout
# Load Baidu picture webpage, stop loading if more than 30 seconds
driver.set_page_load_timeout(30)
try:
    driver.get('http://image.baidu.com/')
except TimeoutException:
    driver.execute_script('window.stop()')

# Enter input box and query
driver.find_element_by_id('kw').send_keys('clams')
driver.find_element_by_class_name('s_search').click()
driver.implicitly_wait(30)


# # Use css_selector to find the first picture and click
# first_img = driver.find_element_by_css_selector('.main_img.img-hover')
# first_img.click()
# driver.implicitly_wait(30)

# # Window handle switch to new window
# driver.switch_to_window(driver.window_handles[1])

# print('开始下载图片.')

# # Download 100 pictures, you can control the number of pictures yourself
# for _ in range(100):

#     try:
#         time.sleep(5)
#         # Click the download image button
#         driver.find_element_by_xpath('//*[@id="toolbar"]/span[7]').click()
#     except TimeoutException:
#         print('Timeout')
#         driver.execute_script('window.stop()')
#         time.sleep(5)
#         driver.find_element_by_xpath('//*[@id="toolbar"]/span[7]').click()
#     except ElementNotVisibleException:
#         print('Element does not exist')
#         driver.refresh()
#         time.sleep(5)
#         driver.find_element_by_xpath('//*[@id="toolbar"]/span[7]').click()
#     except Exception:
#         print('The download of the% d image failed.'%_)

#     time.sleep(2)
#     print('The % d image has been downloaded successfully.' % (_ + 1))
#     # 点击下一张按钮
#     driver.find_elements_by_class_name('img-switch-btn')[1].click()
#     time.sleep(2)


time.sleep(10)
driver.quit()
print('The picture is downloaded!')

Le navigateur est lancé


/home/sully/bin/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: use driver.switch_to.window instead


开始下载图片.
The  1 image has been downloaded successfully.
The  2 image has been downloaded successfully.
The  3 image has been downloaded successfully.
The  4 image has been downloaded successfully.


NoSuchWindowException: Message: Browsing context has been discarded


## ======================================

## ① Scraping static pages

In [17]:
# Source : https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d
# ---------------------------------------------------------------------------------------

import requests
from bs4 import BeautifulSoup
import pandas as pd

# download wikipage
wikipage = "https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependent_territories_by_continent_(data_file)"
result = requests.get(wikipage)

# if successful parse the download into a BeautifulSoup object, which allows easy manipulation 
if result.status_code == 200:
    soup = BeautifulSoup(result.content, "html.parser")
    
# find the object with HTML class wibitable sortable
table = soup.find('table',{'class':'wikitable sortable'})

# loop through all the rows and pull the text
new_table = []
for row in table.find_all('tr')[1:]:
    column_marker = 0
    columns = row.find_all('td')
    new_table.append([column.get_text() for column in columns])
    
df = pd.DataFrame(new_table, columns=['ContinentCode','Alpha2','Alpha3','PhoneCode','Name'])
df['Name'] = df['Name'].str.replace('\n','')
df

,ContinentCode,Alpha2,Alpha3,PhoneCode,Name
0,AS,AF,AFG,004,"Afghanistan, Islamic Republic of"
1,EU,AL,ALB,008,"Albania, Republic of"
2,AN,AQ,ATA,010,Antarctica (the territory South of 60 deg S)
3,AF,DZ,DZA,012,"Algeria, People's Democratic Republic of"
4,OC,AS,ASM,016,American Samoa
...,...,...,...,...,...
258,AF,ZM,ZMB,894,"Zambia, Republic of"
259,AS,XD,null,null,United Nations Neutral Zone
260,AS,XE,null,null,Iraq-Saudi Arabia Neutral Zone
261,AS,XS,null,null,Spratly Islands


## ② Scraping interactive pages

In [ ]:
# adding path to geckodriver to the OS environment variable
os.environ["PATH"] += os.pathsep + os.getcwd()

In [22]:
# Lance le navigateur web
wd = webdriver.Firefox()
wd.get('https://google.com')
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('Dogs')
# Close browser
wd.quit()

## ③ Scraping images from Google

In [106]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions) 
#         -------------------------------
        # Prepare Google Page
        for _ in range(3):
            for __ in range(10):
                # Multiple scrolls needed to show all 400 images
                wd.execute_script("window.scrollBy(0, 1000000)")
                time.sleep(0.2)
            # to load next 400 images
            time.sleep(2.5)
            try:
                wd.find_element_by_xpath("//input[@value='Show more results']").click()
                time.sleep(2.5)
            except Exception as e:
                print("Less images found:"+ str(e))
                break
# -----------------------------------------
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [110]:
def persist_image(folder_path:str, url:str, search_term, img_idx):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

#     try:
    image_file = io.BytesIO(image_content)
    image = Image.open(image_file).convert('RGB')
    file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
    with open(file_path, 'wb') as f:
        image.save(f, "JPEG", quality=85)
#         print(f"SUCCESS - saved {url} - as {file_path}")
#     except Exception as e:
#         print(f"ERROR - Could not save {url} - {e}")

In [113]:
# def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
def search_and_download(search_term:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

#     with webdriver.Chrome(executable_path=driver_path) as wd:
    with webdriver.Firefox() as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
#     for elem in res:
#         persist_image(target_folder,elem)
    for idx, elem in enumerate(res):
        try:
            persist_image(target_folder, elem, search_term, idx)
        except exception:
            print(f"ERROR - search_and_download {exception}")
            continue

In [122]:
# search_terms = ["seafood & fish", "clams"]
# nb_img_by_terms = [3, 101]
# search_terms = ["Oysters"] # SEAFOOD & FISH : Mussels, Octopus, Oysters - OK, Squid, Seafood - OK, cooking fish - OK
search_terms = ["ribs recipe"]   # MEAT : Steak - OK, poultry recipe - OK, ribs recipe - OK,  
# EGGS AND OMELET : omelet, eggs, 
# NATURAL FAT, HIGH-FAT SAUCES : butter, oil, high-fats sauce, bearnaise, garlic butter
# HIGH-FAT DAIRY : butter, cheese
# NUTS : 
# VEGETABLES : Cauliflower, Cabbage, Avocado, Broccoli, Zucchini, Spinach, Asparagus, Kale, Green beans, Brussels sprouts
# FRUITS : Raspberries, Blackberries, Strawberries, Plum, Kiwi, Cherries, Blueberries, Clementine, Cantaloupe, Peach

nb_img_by_terms = [200]

for idx, term in enumerate(search_terms):
    try:
        search_and_download(search_term=term, number_images=nb_img_by_terms[idx])
    except exception:
        print(f"ERROR - global {exception}")
        continue

Less images found:Message: Element <input class="mye4qd" type="button"> could not be scrolled into view

Found: 632 search results. Extracting links from 0:632
Found: 201 image links, done!
